# Proyecto 1 DS — Limpieza 

**Javier Ovalle - 22103**

**José Ángel Morales - 22689**

**Ricardo Morales - 22289**

Link del repositorio: https://github.com/Saiyan-Javi/Proyecto1DS

## 0) Configuración & Metadatos (solo nombres de archivo)

In [73]:
# Archivos SOLO por nombre (rutas relativas)
INPUT_CSV   = "establecimientos_diversificado.csv"
CLEAN_CSV   = "establecimientos_diversificado_limpio.csv"
CAP_CLUSTERS_CSV = "duplicados_capital_merge_v4.csv"
CAP_DETALLE_CSV  = "duplicados_capital_merge_detalle_v4.csv"
CODEBOOK_MD = "Libro_de_Codigos_Proyecto1_v4_3.md"
EXCEL_XLSX  = "salidas_proyecto1_v4_3.xlsx"
BITACORA_CSV= "bitacora_limpieza_v4_3.csv"


## 1) Imports

In [74]:
import re, os, unicodedata
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 160)

## 2) Carga del CSV consolidado (+ snapshot crudo)

In [75]:
# Cargar TODO como texto para no perder ceros a la izquierda ni formatos
df = pd.read_csv(INPUT_CSV, dtype=str)
df.columns = [c.upper() for c in df.columns]
df_raw = df.copy(deep=True)
df.shape, df.columns.tolist()[:20]

((6599, 18),
 ['CODIGO',
  'DISTRITO',
  'DEPARTAMENTO',
  'MUNICIPIO',
  'ESTABLECIMIENTO',
  'DIRECCION',
  'TELEFONO',
  'SUPERVISOR',
  'DIRECTOR',
  'NIVEL',
  'SECTOR',
  'AREA',
  'STATUS',
  'MODALIDAD',
  'JORNADA',
  'PLAN',
  'DEPARTAMENTAL',
  'DEPARTAMENTO_ORIGEN'])

## 3) Utilidades de normalización y parsing

In [76]:
import unicodedata

def strip_accents(s: str) -> str:
    if not isinstance(s, str): return s
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def collapse_spaces_and_hyphens(s: str) -> str:
    if not isinstance(s, str): return s
    s = s.replace("—","-").replace("–","-").replace("−","-").replace("­","")
    s = re.sub(r"[ \t\u00A0]+", " ", s)
    s = re.sub(r"-{2,}", "-", s)
    s = re.sub(r"\s*-\s*", " - ", s)
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip()

def normalize_text_basic(s: str) -> str:
    if not isinstance(s, str): return s
    s = s.strip().replace('"', "").replace("“","").replace("”","").replace("´", "'")
    s = strip_accents(s).upper()
    s = collapse_spaces_and_hyphens(s)
    return s

def remove_leading_bullets(s: str) -> str:
    if not isinstance(s, str): return s
    return re.sub(r"^\s*[-•]+\s*", "", s)

def expand_address_abbrev(s: str) -> str:
    if not isinstance(s, str): return s
    txt = " " + s + " "
    rules = [
        (r"\bAV[\.]?\b", " AVENIDA "), (r"\bAVE[\.]?\b", " AVENIDA "), (r"\bAVDA[\.]?\b", " AVENIDA "),
        (r"\bBLVD[\.]?\b", " BOULEVARD "), (r"\bCALZ[\.]?\b", " CALZADA "),
        (r"\bCOL[\.]?\b", " COLONIA "), (r"\bCOND[\.]?\b", " CONDOMINIO "), (r"\bRES[\.]?\b", " RESIDENCIAL "),
        (r"\bZ[\.]?\b(?=\s*\d)", " ZONA "), (r"\bZONA\b\s*(?=\d)", " ZONA "),
        (r"\bKM[\.]?\b", " KM "), (r"\bNO[\.]?\b", " NUMERO "), (r"\bN[\.]?\b(?=\s*\d)", " NUMERO "), (r"\b#\b", " NUMERO "),
        (r"\bEDIF[\.]?\b", " EDIFICIO "), (r"\bTORRE\b", " TORRE "),
        (r"\b2DA\b", " 2A "), (r"\b3RA\b", " 3A ")
    ]
    for pat, rep in rules:
        txt = re.sub(pat, rep, txt, flags=re.IGNORECASE)
    txt = collapse_spaces_and_hyphens(txt)
    return txt

def normalize_name(s: str) -> str:
    if not isinstance(s, str): return s
    s = remove_leading_bullets(s)
    s = normalize_text_basic(s)
    return s

def fix_ruta_ocr_general(s: str) -> str:
    if not isinstance(s, str): return s
    txt = s
    txt = re.sub(r"\b(RUTA\s+)[ILil]\s*(\d)\b", r"\g<1>1\2", txt)
    txt = re.sub(r"\b(RUTA\s+)[Oo]\s*(\d)\b", r"\g<1>0\2", txt)
    return txt

def clean_phone_field(s: str):
    if not isinstance(s, str): return None
    nums = re.findall(r"\d{8}", s)
    nums = [n for n in nums if not re.fullmatch(r"0{8}", n)]
    seen, unique = set(), []
    for n in nums:
        if n not in seen:
            seen.add(n); unique.append(n)
    return " / ".join(unique) if unique else None

def extract_zona_token(text: str):
    if not isinstance(text, str): return None
    m = re.search(r"\bZONA\s*(\d{1,2})\b", text)
    if m:
        return (m.group(1).lstrip("0") or "0")
    return None

def parse_address_components(addr: str):
    if not isinstance(addr, str) or not addr.strip():
        return {}, None, ""
    a = expand_address_abbrev(normalize_text_basic(addr))
    a = fix_ruta_ocr_general(a)
    zona = extract_zona_token(a)
    a_no_zona = re.sub(r"\bZONA\s*\d{1,2}\b", "", a).strip()
    a_no_zona = collapse_spaces_and_hyphens(a_no_zona)

    comp = {}
    m = re.search(r"\bAVENIDA\s+([0-9]{1,2}[A-Z]?)\b", a_no_zona) or re.search(r"\b([0-9]{1,2}[A-Z]?)\s+AVENIDA\b", a_no_zona)
    if m: comp["AVENIDA"] = m.group(1)
    m = re.search(r"\bCALLE\s+([0-9]{1,2}[A-Z]?)\b", a_no_zona) or re.search(r"\b([0-9]{1,2}[A-Z]?)\s+CALLE\b", a_no_zona)
    if m: comp["CALLE"] = m.group(1)
    m = re.search(r"\bKM\s*([0-9]+(?:\.[0-9]+)?)\b", a_no_zona)
    if m: comp["KM"] = m.group(1)
    m = re.search(r"\bNUMERO\s*([0-9A-Z\-\/]+)\b", a_no_zona)
    if m: comp["NUMERO"] = m.group(1)

    nominativos = ["COLONIA","RESIDENCIAL","CONDOMINIO","EDIFICIO","TORRE","BARRIO","SECTOR","BOULEVARD","CALZADA","CARRETERA","RUTA"]
    nom_found, rem = [], a_no_zona
    for key in nominativos:
        m = re.search(rf"\b{key}\b\s*([A-Z0-9\-\.\s]+)", rem)
        if m:
            val = m.group(1).strip()
            for stop in nominativos + ["AVENIDA","CALLE","KM","NUMERO"]:
                val = re.split(rf"\b{stop}\b", val)[0]
            val = val.strip(" ,.;-")
            if val:
                nom_found.append(f"{key} {val}")
                rem = re.sub(rf"\b{key}\b\s*[A-Z0-9\-\.\s]+", "", rem, count=1).strip()

    parts = []
    if "AVENIDA" in comp: parts.append(f"AVENIDA {comp['AVENIDA']}")
    if "CALLE"   in comp: parts.append(f"CALLE {comp['CALLE']}")
    if "KM"      in comp: parts.append(f"KM {comp['KM']}")
    if "NUMERO"  in comp: parts.append(f"NUMERO {comp['NUMERO']}")
    if nom_found: parts.extend(nom_found)

    resto = rem.strip(",; ")
    resto = collapse_spaces_and_hyphens(resto)
    for p in parts + ([f"ZONA {zona}"] if zona else []):
        if p:
            resto = re.sub(re.escape(p), "", resto).strip(",; ")
    direccion_std = ", ".join([p for p in parts if p]) if parts else a_no_zona
    if resto and resto not in direccion_std:
        direccion_std = ", ".join([direccion_std, resto]) if direccion_std else resto
    direccion_std = collapse_spaces_and_hyphens(direccion_std)
    direccion_std = re.sub(r"\s{2,}", " ", direccion_std).strip(" ,")

    return comp, zona, direccion_std


## 3A) Estado inicial de los datos (CRUDO)

In [77]:
na_por_col = df_raw.isna().sum().sort_values(ascending=False)

placeholders = {"", "nan", "None", "-", "--", "—"}
ph_counts = {}
for col in df_raw.columns:
    s = df_raw[col].astype(str).str.strip()
    ph_counts[col] = s.isin(placeholders).sum()
ph_counts = pd.Series(ph_counts).sort_values(ascending=False)

bullets_est = df_raw.get("ESTABLECIMIENTO", pd.Series(dtype=str)).fillna("").str.match(r"^\s*[-•]+\s*").sum()
bullets_dir = df_raw.get("DIRECCION", pd.Series(dtype=str)).fillna("").str.match(r"^\s*[-•]+\s*").sum()

pat_zona = r"\bZ(\.|\s)*\s*\d{1,2}\b|\bZONA\s*\d{1,2}\b"
zona_en_dir = df_raw.get("DIRECCION", pd.Series(dtype=str)).fillna("").str.contains(pat_zona, regex=True, case=False).sum()
zona_en_depto = df_raw.get("DEPARTAMENTO", pd.Series(dtype=str)).fillna("").str.contains(pat_zona, regex=True, case=False).sum()
zona_en_muni = df_raw.get("MUNICIPIO", pd.Series(dtype=str)).fillna("").str.contains(pat_zona, regex=True, case=False).sum()

pat_cap = r"\bCIUDAD\s+CAPITAL\b"
capital_en_depto = df_raw.get("DEPARTAMENTO", pd.Series(dtype=str)).fillna("").str.contains(pat_cap, regex=True, case=False).sum()
capital_en_muni  = df_raw.get("MUNICIPIO", pd.Series(dtype=str)).fillna("").str.contains(pat_cap, regex=True, case=False).sum()

tel_raw = df_raw.get("TELEFONO", pd.Series(dtype=str)).fillna("")
has_digits = tel_raw.str.contains(r"\d")
has_group8 = tel_raw.str.contains(r"\b\d{8}\b")
tel_formato_raro = (has_digits & ~has_group8).sum()

pat_ocr = r"\bRUTA\s+[ILiLoO]\s*\d\b"
ruta_ocr = df_raw.get("DIRECCION", pd.Series(dtype=str)).fillna("").str.contains(pat_ocr, regex=True).sum()

def _norm_crudo(x):
    x = str(x) if pd.notna(x) else ""
    x = x.strip().upper()
    return x

key_cruda = (
    df_raw.get("ESTABLECIMIENTO","").map(_norm_crudo) + "|" +
    df_raw.get("DIRECCION","").map(_norm_crudo) + "|" +
    df_raw.get("MUNICIPIO","").map(_norm_crudo) + "|" +
    df_raw.get("DEPARTAMENTO","").map(_norm_crudo)
)
dup_crudos_total = (key_cruda.duplicated(keep=False)).sum()
dup_crudos_clusters = key_cruda[key_cruda.duplicated(keep=False)].value_counts().shape[0]

resumen_crudo = pd.DataFrame({
    "métrica": [
        "NA totales (suma por col)", "placeholders (suma máx por col)",
        "bullets en ESTABLECIMIENTO", "bullets en DIRECCION",
        "ZONA en DIRECCION", "ZONA en DEPARTAMENTO", "ZONA en MUNICIPIO",
        "CIUDAD CAPITAL en DEPARTAMENTO", "CIUDAD CAPITAL en MUNICIPIO",
        "TELEFONO con formato raro", "Patrones OCR en RUTA (DIRECCION)",
        "Duplicados crudos (filas)", "Duplicados crudos (clusters)"
    ],
    "valor": [
        int(na_por_col.sum()), int(ph_counts.max()) if len(ph_counts)>0 else 0,
        int(bullets_est), int(bullets_dir),
        int(zona_en_dir), int(zona_en_depto), int(zona_en_muni),
        int(capital_en_depto), int(capital_en_muni),
        int(tel_formato_raro), int(ruta_ocr),
        int(dup_crudos_total), int(dup_crudos_clusters)
    ]
})
na_por_col = na_por_col.to_frame("NA_por_columna")
ph_counts = ph_counts.to_frame("placeholders_por_columna")

resumen_crudo, na_por_col.head(15), ph_counts.head(15)

C:\Users\ricar\AppData\Local\Temp\ipykernel_18336\376297493.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  zona_en_dir = df_raw.get("DIRECCION", pd.Series(dtype=str)).fillna("").str.contains(pat_zona, regex=True, case=False).sum()
C:\Users\ricar\AppData\Local\Temp\ipykernel_18336\376297493.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  zona_en_depto = df_raw.get("DEPARTAMENTO", pd.Series(dtype=str)).fillna("").str.contains(pat_zona, regex=True, case=False).sum()
C:\Users\ricar\AppData\Local\Temp\ipykernel_18336\376297493.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  zona_en_muni = df_raw.get("MUNICIPIO", pd.Series(dtype=str)).fillna("").str.contains(pat_zona, regex=True, case=False).sum()


(                             métrica  valor
 0          NA totales (suma por col)     74
 1    placeholders (suma máx por col)     46
 2         bullets en ESTABLECIMIENTO      1
 3               bullets en DIRECCION      0
 4                  ZONA en DIRECCION   3050
 5               ZONA en DEPARTAMENTO      0
 6                  ZONA en MUNICIPIO    866
 7     CIUDAD CAPITAL en DEPARTAMENTO    866
 8        CIUDAD CAPITAL en MUNICIPIO      0
 9          TELEFONO con formato raro     25
 10  Patrones OCR en RUTA (DIRECCION)      1
 11         Duplicados crudos (filas)   2141
 12      Duplicados crudos (clusters)    855,
                  NA_por_columna
 TELEFONO                     46
 DIRECTOR                     26
 DIRECCION                     2
 CODIGO                        0
 DISTRITO                      0
 DEPARTAMENTO                  0
 ESTABLECIMIENTO               0
 MUNICIPIO                     0
 SUPERVISOR                    0
 NIVEL                         0
 SECTO

## 4) Limpieza básica (+ justificación)

In [78]:
obj_cols = [c for c in df.columns if df[c].dtype == "object"]
df[obj_cols] = df[obj_cols].apply(lambda s: s.map(lambda x: collapse_spaces_and_hyphens(x) if isinstance(x, str) else x))
df[obj_cols] = df[obj_cols].replace({"": pd.NA, "nan": pd.NA, "None": pd.NA, "-": pd.NA, "--": pd.NA, "—": pd.NA})

for col in ["ESTABLECIMIENTO","DIRECCION","TELEFONO"]:
    if col in df.columns and f"{col}_ORIG" not in df.columns:
        df[f"{col}_ORIG"] = df[col]

for col in ["ESTABLECIMIENTO","DIRECCION"]:
    if col in df.columns:
        df[col] = df[col].map(remove_leading_bullets)

if "ESTABLECIMIENTO" in df.columns:
    df["ESTABLECIMIENTO"] = df["ESTABLECIMIENTO"].map(normalize_name)

if "TELEFONO" in df.columns:
    df["TELEFONO"] = df["TELEFONO"].map(clean_phone_field)
    df["TELEFONO_VALIDO"] = df["TELEFONO"].apply(lambda x: bool(x) if pd.notna(x) else False)

for col in ["DEPARTAMENTO","MUNICIPIO","SECTOR","AREA","STATUS","MODALIDAD","JORNADA","PLAN","NIVEL"]:
    if col in df.columns:
        df[col] = df[col].map(lambda x: normalize_text_basic(x) if isinstance(x, str) else x)

df.shape

(6599, 22)

## 5) Dirección → ZONA y DIRECCION_STD

In [79]:
import re

# Funciones auxiliares necesarias
def normalize_text_basic(text: str) -> str:
    text = text.upper()
    text = re.sub(r'[ÁÀÂÃÄ]', 'A', text)
    text = re.sub(r'[ÉÈÊË]', 'E', text)
    text = re.sub(r'[ÍÌÎÏ]', 'I', text)
    text = re.sub(r'[ÓÒÔÕÖ]', 'O', text)
    text = re.sub(r'[ÚÙÛÜ]', 'U', text)
    text = re.sub(r'[Ç]', 'C', text)
    text = re.sub(r'[Ñ]', 'N', text)
    text = re.sub(r'[^A-Z0-9\s\.\-]', ' ', text)
    return text

def expand_address_abbrev(text: str) -> str:
    expansions = {
        r'\bAV\b': 'AVENIDA',
        r'\bCL\b': 'CALLE',
        r'\bCTRA\b': 'CARRETERA',
        r'\bBLVD\b': 'BOULEVARD',
        r'\bCOL\b': 'COLONIA',
        r'\bRES\b': 'RESIDENCIAL',
        r'\bCOND\b': 'CONDOMINIO',
        r'\bEDIF\b': 'EDIFICIO',
        r'\bTORRE\b': 'TORRE',
        r'\bSECT\b': 'SECTOR',
        r'\bCALZ\b': 'CALZADA',
        r'\bDIAG\b': 'DIAGONAL',
        r'\bALD\b': 'ALDEA',
        r'\bKM\b': 'KM',
        r'\bNO\b': 'NUMERO',
    }
    for abbr, full in expansions.items():
        text = re.sub(abbr, full, text, flags=re.IGNORECASE)
    return text

def fix_ruta_ocr_general(text: str) -> str:
    return text

def extract_zona_token(text: str):
    m = re.search(r'\bZONA\s*(\d{1,2})\b', text, re.IGNORECASE)
    if m:
        return m.group(1)
    return None

def collapse_spaces_and_hyphens(text: str) -> str:
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s*-\s*', '-', text)
    return text

# Función parse modificada para manejar correctamente calles y rangos
def parse_address_components(addr: str):
    if not isinstance(addr, str) or not addr.strip():
        return {}, None, ""

    a = expand_address_abbrev(normalize_text_basic(addr))
    a = fix_ruta_ocr_general(a)
    zona = extract_zona_token(a)
    a_no_zona = re.sub(r"\bZONA\s*\d{1,2}\b", "", a, flags=re.IGNORECASE).strip()
    a_no_zona = collapse_spaces_and_hyphens(a_no_zona)

    comp = {}

    # Patrones mejorados para AVENIDA y CALLE
    aven_pat = r"\b(\d{1,2}[A-Z]{0,2})\.?\s*AVENIDA\b|\bAVENIDA\s*(\d{1,2}[A-Z]{0,2})\.?"
    calle_pat = r"\b(\d{1,2}[A-Z]{0,2})\.?\s*CALLE\b|\bCALLE\s*(\d{1,2}[A-Z]{0,2})\.?"

    # Capturar AVENIDA
    m = re.search(aven_pat, a_no_zona, re.IGNORECASE)
    if m:
        comp["AVENIDA"] = (m.group(1) or m.group(2)).upper()

    # Capturar CALLE
    m = re.search(calle_pat, a_no_zona, re.IGNORECASE)
    if m:
        comp["CALLE"] = (m.group(1) or m.group(2)).upper()

    # Capturar KM
    m = re.search(r"\bKM\s*(\d+(?:\.\d+)?)\b", a_no_zona, re.IGNORECASE)
    if m:
        comp["KM"] = m.group(1)

    # Lógica mejorada para interpretación de rangos
    range_pat = r"(\d{1,2}[A-Z]?)\s*-\s*(\d{1,3}[A-Z]?)"

    if "AVENIDA" in comp and "CALLE" not in comp:
        # Buscar rango después de AVENIDA (CALLE - NUMERO)
        post_aven = a_no_zona.split("AVENIDA", 1)[-1]
        m = re.search(range_pat, post_aven)
        if m:
            comp["CALLE"] = m.group(1)
            comp["NUMERO"] = m.group(2)

    elif "CALLE" in comp and "NUMERO" not in comp:
        # Buscar rango después de CALLE (AVENIDA - NUMERO)
        post_calle = a_no_zona.split("CALLE", 1)[-1]
        m = re.search(range_pat, post_calle)
        if m:
            if "AVENIDA" not in comp:
                comp["AVENIDA"] = m.group(1)
            comp["NUMERO"] = m.group(2)
    else:
        # Buscar rango general para NUMERO
        m = re.search(range_pat, a_no_zona)
        if m and "NUMERO" not in comp:
            comp["NUMERO"] = f"{m.group(1)}-{m.group(2)}"

    # Nominativos
    nominativos = ["COLONIA","RESIDENCIAL","CONDOMINIO","EDIFICIO","TORRE","BARRIO","SECTOR","BOULEVARD","CALZADA","CARRETERA","RUTA","DIAGONAL","ALDEA"]
    nom_found = []
    rem = a_no_zona

    for key in nominativos:
        # Buscar hasta el siguiente nominativo o fin de cadena
        pattern = rf"\b{key}\s+([^,;]+?)(?=\b(?:{'|'.join(nominativos)})\b|$)"
        m = re.search(pattern, rem, re.IGNORECASE)
        if not m:
            # Buscar sin lookahead
            m = re.search(rf"\b{key}\s+([^,;]+)", rem, re.IGNORECASE)
        if m:
            val = m.group(1).strip()
            # Detener en otros componentes importantes
            for stop in nominativos + ["AVENIDA","CALLE","KM","NUMERO"]:
                if re.search(rf"\b{stop}\b", val, re.IGNORECASE):
                    val = re.split(rf"\b{stop}\b", val, flags=re.IGNORECASE)[0].strip()
            val = val.strip(' ,.;-')
            if val:
                nom_found.append(f"{key.upper()} {val}")
                rem = re.sub(re.escape(m.group(0)), "", rem).strip()

    # Construir STD en orden lógico: CALLE, AVENIDA, NUMERO
    parts = []
    if "CALLE" in comp:
        parts.append(f"CALLE {comp['CALLE']}")
    if "AVENIDA" in comp:
        parts.append(f"AVENIDA {comp['AVENIDA']}")
    if "KM" in comp:
        parts.append(f"KM {comp['KM']}")
    if "NUMERO" in comp:
        parts.append(f"NUMERO {comp['NUMERO']}")
    if nom_found:
        parts.extend(nom_found)

    # Procesar el resto de la dirección
    resto = rem.strip(",; ")
    resto = collapse_spaces_and_hyphens(resto)
    for p in parts:
        resto = re.sub(re.escape(p), "", resto, flags=re.IGNORECASE).strip(",; ")

    if resto:
        parts.append(resto)

    direccion_std = ", ".join(parts)
    direccion_std = collapse_spaces_and_hyphens(direccion_std)

    return comp, zona, direccion_std

# Procesamiento del DataFrame
# Respaldos originales
df["DIRECCION_ORIG"] = df["DIRECCION"].copy()

# Inicializar columnas
comp_cols = ["DIR_AVENIDA","DIR_CALLE","DIR_KM","DIR_NUMERO"]
for cc in comp_cols:
    df[cc] = None
df["ZONA"] = None
df["DIRECCION_STD"] = None

# Aplicar la función a cada fila
for idx, row in df.iterrows():
    addr = row.get("DIRECCION")
    comp, zona, direccion_std = parse_address_components(addr)
    for k, v in comp.items():
        df.at[idx, f"DIR_{k}"] = v
    df.at[idx, "ZONA"] = zona
    df.at[idx, "DIRECCION_STD"] = direccion_std

# Mostrar resultados
df[["DIRECCION_ORIG", "DIRECCION_STD", "DIR_AVENIDA", "DIR_CALLE", "DIR_NUMERO", "ZONA"]].sample(5)

,DIRECCION_ORIG,DIRECCION_STD,DIR_AVENIDA,DIR_CALLE,DIR_NUMERO,ZONA
1316,31 AVE. 14 - 26 COLONIA CIUDAD DE PLATA II,"NUMERO 14-26, COLONIA CIUDAD DE PLATA II, 31 AVE. 14-26",None,None,14-26,None
6206,3A. CALLE NO. 214 ZONA 2 BARRIO LA UNION,"CALLE 3A, BARRIO LA UNION, 3A. CALLE NUMERO. 214",None,3A,None,2
5523,7A. CALLE 12 - 99 ZONA 4,"CALLE 7A, AVENIDA 12, NUMERO 99, 7A. CALLE 12-99",12,7A,99,4
1473,CALZADA AGUILAR BATRES 29 - 41 Y 1A. AVENIDA 29 - 26 COLONIA EL CARMEN,"CALLE 29, AVENIDA 1A, NUMERO 26, COLONIA EL CARMEN, CALZADA AGUILAR BATRES 29-41 Y 1A",1A,29,26,None
5254,"7A. Y 8A. CALLE, AVENIDA ""COATEPEQUE""","CALLE 8A, 7A. Y 8A. CALLE AVENIDA COATEPEQUE",None,8A,None,None


## 6) Normalización de 'CIUDAD CAPITAL' y zonas

In [80]:
def normalize_capital_and_zone_row(row):
    dept = str(row.get("DEPARTAMENTO", "") or "")
    muni = str(row.get("MUNICIPIO", "") or "")
    is_capital = bool(re.search(r"\bCIUDAD\s+CAPITAL\b", dept) or re.search(r"\bCIUDAD\s+CAPITAL\b", muni))
    z_depto = extract_zona_token(dept)
    z_muni  = extract_zona_token(muni)
    zona = row.get("ZONA", pd.NA)
    zona = zona if pd.notna(zona) else (z_depto or z_muni)
    if is_capital or z_depto is not None or z_muni is not None:
        row["DEPARTAMENTO"] = "GUATEMALA"
        row["MUNICIPIO"] = "GUATEMALA"
        row["ZONA"] = zona
    return row

df = df.apply(normalize_capital_and_zone_row, axis=1)
df["ZONA"] = df["ZONA"].where(df["ZONA"].notna() & df["ZONA"].astype(str).str.strip().ne(""), pd.NA)

df[["DEPARTAMENTO","MUNICIPIO","ZONA"]].head(10)

,DEPARTAMENTO,MUNICIPIO,ZONA
0,ALTA VERAPAZ,COBAN,8
1,ALTA VERAPAZ,COBAN,<NA>
2,ALTA VERAPAZ,COBAN,6
3,ALTA VERAPAZ,COBAN,2
4,ALTA VERAPAZ,COBAN,11
5,ALTA VERAPAZ,COBAN,3
6,ALTA VERAPAZ,COBAN,4
7,ALTA VERAPAZ,COBAN,8
8,ALTA VERAPAZ,COBAN,1
9,ALTA VERAPAZ,COBAN,4


## 7) Duplicados en Ciudad de Guatemala

In [81]:
def _norm(s): return normalize_text_basic(s) if isinstance(s, str) else ""

df["__KEY_MERGE_CAP__"] = df.apply(lambda r: "|".join([
    _norm(r.get("ESTABLECIMIENTO","")),
    _norm(r.get("DIRECCION_STD","")),
    _norm(r.get("MUNICIPIO","")),
    _norm(r.get("DEPARTAMENTO","")),
    _norm(r.get("JORNADA","")),
    _norm(r.get("PLAN","")),
    _norm(r.get("TELEFONO","")),
]), axis=1)

mask_capital = (df["DEPARTAMENTO"]=="GUATEMALA") & (df["MUNICIPIO"]=="GUATEMALA")
grp_sizes = df[mask_capital].groupby("__KEY_MERGE_CAP__")["__KEY_MERGE_CAP__"].transform("size") if mask_capital.any() else pd.Series([], dtype=int)

df["DUP_CAPITAL_MERGE"] = False
if mask_capital.any():
    df.loc[mask_capital & grp_sizes.gt(1), "DUP_CAPITAL_MERGE"] = True

dup_cap_groups = df.loc[df["DUP_CAPITAL_MERGE"]].groupby("__KEY_MERGE_CAP__")
if len(dup_cap_groups) > 0:
    key_has_diff_codes = dup_cap_groups["CODIGO"].apply(lambda s: len(set(s.dropna().astype(str))) > 1)
    key_has_diff_codes = key_has_diff_codes.reindex(df["__KEY_MERGE_CAP__"]).fillna(False).values
else:
    key_has_diff_codes = [False]*len(df)
df["DUP_CAPITAL_CODIGOS_DISTINTOS"] = key_has_diff_codes

df.loc[df["DUP_CAPITAL_MERGE"], ["ESTABLECIMIENTO","DIRECCION_STD","ZONA","MUNICIPIO","DEPARTAMENTO","JORNADA","PLAN","CODIGO","DUP_CAPITAL_CODIGOS_DISTINTOS"]].head(20)

C:\Users\ricar\AppData\Local\Temp\ipykernel_18336\1907362373.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  key_has_diff_codes = key_has_diff_codes.reindex(df["__KEY_MERGE_CAP__"]).fillna(False).values


,ESTABLECIMIENTO,DIRECCION_STD,ZONA,MUNICIPIO,DEPARTAMENTO,JORNADA,PLAN,CODIGO,DUP_CAPITAL_CODIGOS_DISTINTOS
839,LICEO TECNOLOGICO GUATEMALA DE LA ASUNCION,"CALLE 3A, AVENIDA 7, NUMERO 64, 3A. CALLE 7-64",1,GUATEMALA,GUATEMALA,DOBLE,FIN DE SEMANA,00 - 01 - 0197 - 46,True
848,LICEO TECNOLOGICO GUATEMALA DE LA ASUNCION,"CALLE 3A, AVENIDA 7, NUMERO 64, 3A. CALLE 7-64",1,GUATEMALA,GUATEMALA,DOBLE,FIN DE SEMANA,00 - 01 - 0218 - 46,True
868,LICEO HISPANOAMERICANO NO. 2,"CALLE 2A, AVENIDA 7, NUMERO 11, 2A. CALLE 7-11",1,GUATEMALA,GUATEMALA,MATUTINA,DIARIO(REGULAR),00 - 01 - 0318 - 46,True
872,LICEO TECNICO INTEGRAL SANTA TERESA DE JESUS,"CALLE 12, AVENIDA 10, NUMERO 37, 12 CALLE 10-37",1,GUATEMALA,GUATEMALA,MATUTINA,DIARIO(REGULAR),00 - 01 - 0353 - 46,True
875,INSTITUTO PARTICULAR MIXTO CIENCIA Y DESARROLLO,"CALLE 10, AVENIDA 3, NUMERO 15, 10 CALLE 3-15",1,GUATEMALA,GUATEMALA,DOBLE,FIN DE SEMANA,00 - 01 - 0366 - 46,True
877,LICEO TECNICO INTEGRAL SANTA TERESA DE JESUS,"CALLE 12, AVENIDA 10, NUMERO 37, 12 CALLE 10-37",1,GUATEMALA,GUATEMALA,DOBLE,FIN DE SEMANA,00 - 01 - 0376 - 46,True
878,LICEO TECNICO INTEGRAL SANTA TERESA DE JESUS,"CALLE 12, AVENIDA 10, NUMERO 37, 12 CALLE 10-37",1,GUATEMALA,GUATEMALA,DOBLE,FIN DE SEMANA,00 - 01 - 0377 - 46,True
888,LICEO CULTURAL ESPANOL,"CALLE 3, AVENIDA 4A, NUMERO 56, 4A. AVENIDA 3-56",1,GUATEMALA,GUATEMALA,DOBLE,FIN DE SEMANA,00 - 01 - 0458 - 46,True
889,LICEO CULTURAL ESPANOL,"CALLE 3, AVENIDA 4A, NUMERO 56, 4A. AVENIDA 3-56",1,GUATEMALA,GUATEMALA,DOBLE,FIN DE SEMANA,00 - 01 - 0460 - 46,True
900,LICEO MONTE SINAI,"NUMERO 7-34, 3A. AVE. 7-34",1,GUATEMALA,GUATEMALA,MATUTINA,FIN DE SEMANA,00 - 01 - 0520 - 46,True


## 8) Guardar CSVs de salida

In [82]:
df.to_csv(CLEAN_CSV, index=False, encoding="utf-8")

cap_dups = df.loc[df["DUP_CAPITAL_MERGE"], ["__KEY_MERGE_CAP__","CODIGO","ESTABLECIMIENTO","DIRECCION_STD","ZONA","TELEFONO","JORNADA","PLAN","MUNICIPIO","DEPARTAMENTO","DUP_CAPITAL_CODIGOS_DISTINTOS"]]

cluster_rows = []
for key, sub in cap_dups.groupby("__KEY_MERGE_CAP__"):
    cods = sorted(set(sub["CODIGO"].dropna().astype(str)))
    cluster_rows.append({
        "KEY": key,
        "ESTABLECIMIENTO": sub["ESTABLECIMIENTO"].iloc[0],
        "DIRECCION_STD": sub["DIRECCION_STD"].iloc[0],
        "ZONA": sub["ZONA"].iloc[0],
        "JORNADA": sub["JORNADA"].iloc[0],
        "PLAN": sub["PLAN"].iloc[0],
        "TELEFONO": sub["TELEFONO"].iloc[0],
        "MUNICIPIO": sub["MUNICIPIO"].iloc[0],
        "DEPARTAMENTO": sub["DEPARTAMENTO"].iloc[0],
        "N_FILAS": len(sub),
        "CODIGOS": " | ".join(cods),
        "CODIGOS_DISTINTOS": (len(cods) > 1)
    })
cap_clusters = pd.DataFrame(cluster_rows).sort_values(["ESTABLECIMIENTO","ZONA","DIRECCION_STD","JORNADA","PLAN"]) if cluster_rows else pd.DataFrame(columns=["KEY","ESTABLECIMIENTO","DIRECCION_STD","ZONA","JORNADA","PLAN","TELEFONO","MUNICIPIO","DEPARTAMENTO","N_FILAS","CODIGOS","CODIGOS_DISTINTOS"])

cap_clusters.to_csv(CAP_CLUSTERS_CSV, index=False, encoding="utf-8")
cap_dups.drop(columns=["__KEY_MERGE_CAP__"]).to_csv(CAP_DETALLE_CSV, index=False, encoding="utf-8")

CLEAN_CSV, CAP_CLUSTERS_CSV, CAP_DETALLE_CSV

('establecimientos_diversificado_limpio.csv',
 'duplicados_capital_merge_v4.csv',
 'duplicados_capital_merge_detalle_v4.csv')

## 9) Comparativa ORIG vs Limpio (QC)

In [83]:
def cambios(col):
    orig = f"{col}_ORIG"
    if orig not in df.columns: 
        return pd.DataFrame(columns=[col, orig])
    mask = df[orig].fillna("") != df[col].fillna("")
    return df.loc[mask, [orig, col]].rename(columns={orig: "ORIG", col: "LIMPIO"}).head(20)

c_est = cambios("ESTABLECIMIENTO")
c_dir = cambios("DIRECCION")
c_tel = cambios("TELEFONO")

resumen_cambios = {
    "ESTABLECIMIENTO_cambiados": int((df.get("ESTABLECIMIENTO_ORIG","").fillna("") != df.get("ESTABLECIMIENTO","").fillna("")).sum()) if "ESTABLECIMIENTO_ORIG" in df.columns else 0,
    "DIRECCION_cambiadas": int((df.get("DIRECCION_ORIG","").fillna("") != df.get("DIRECCION","").fillna("")).sum()) if "DIRECCION_ORIG" in df.columns else 0,
    "TELEFONO_cambiados": int((df.get("TELEFONO_ORIG","").fillna("") != df.get("TELEFONO","").fillna("")).sum()) if "TELEFONO_ORIG" in df.columns else 0,
    "ZONA_con_valor": int(df["ZONA"].notna().sum()) if "ZONA" in df.columns else 0
}
resumen_cambios, c_est, c_dir, c_tel

({'ESTABLECIMIENTO_cambiados': 3047,
  'DIRECCION_cambiadas': 0,
  'TELEFONO_cambiados': 60,
  'ZONA_con_valor': 3764},
                                                                                                          ORIG  \
 2                                                                                     COLEGIO "LA INMACULADA"   
 6                                                                                      LICEO "MODERNO LATINO"   
 8                                                                           COLEGIO DE INFORMATICA "CENINFAV"   
 11  INSTITUTO NORMAL PREPRIMARIA BILINGUE ADSCRITO AL INSTITUTO NORMAL MIXTO DEL NORTE "EMILIO ROSALES PONCE"   
 14                                                               INSTITUTO MIXTO PRIVADO "SALUD Y DESARROLLO"   
 23                                                             COLEGIO PARTICULAR MIXTO "ANTON DE MONTESINOS"   
 26                                                        CENTRO DE FORMACIÓN INT

## 10) Bitácora de limpieza

In [91]:
bitacora = []

def add_step(nombre, descripcion, métricas: dict):
    bitacora.append({
        "paso": nombre,
        "descripcion": descripcion,
        **{f"m_{k}": v for k, v in métricas.items()}
    })

add_step("Limpieza base",
         "Normalización de espacios/guiones; estandarización de NA; remoción de bullets; categóricas a mayúsculas; extracción tel.",
         {
             "est_cambiados": resumen_cambios.get("ESTABLECIMIENTO_cambiados", 0),
             "dir_cambiadas": resumen_cambios.get("DIRECCION_cambiadas", 0),
             "tel_cambiados": resumen_cambios.get("TELEFONO_cambiados", 0)
         })

add_step("Dirección/ZONA",
         "Extracción de ZONA desde DIRECCION/DEPTO/MUNI, y creación de DIRECCION_STD en orden canónico.",
         {
             "zona_con_valor": resumen_cambios.get("ZONA_con_valor", 0)
         })

add_step("Capital",
         "Normalización de CIUDAD CAPITAL a GUATEMALA/GUATEMALA y traslado de ZONA al campo correcto.",
         {
             "capital_registros": int(((df['DEPARTAMENTO']=='GUATEMALA') & (df['MUNICIPIO']=='GUATEMALA')).sum())
         })

add_step("Duplicados Capital",
         "Marcado de clusters por posible doble unión (sin eliminar filas).",
         {
             "clusters": int(pd.read_csv(CAP_CLUSTERS_CSV, dtype=str).shape[0]) if os.path.exists(CAP_CLUSTERS_CSV) else 0,
             "filas_dup": int(pd.read_csv(CAP_DETALLE_CSV, dtype=str).shape[0]) if os.path.exists(CAP_DETALLE_CSV) else 0
         })

bitacora_df = pd.DataFrame(bitacora)
bitacora_df.to_csv(BITACORA_CSV, index=False, encoding="utf-8")
BITACORA_CSV, bitacora_df.head()

('bitacora_limpieza_v4_3.csv',
                  paso  \
 0       Limpieza base   
 1      Dirección/ZONA   
 2             Capital   
 3  Duplicados Capital   
 
                                                                                                                 descripcion  \
 0  Normalización de espacios/guiones; estandarización de NA; remoción de bullets; categóricas a mayúsculas; extracción tel.   
 1                             Extracción de ZONA desde DIRECCION/DEPTO/MUNI, y creación de DIRECCION_STD en orden canónico.   
 2                               Normalización de CIUDAD CAPITAL a GUATEMALA/GUATEMALA y traslado de ZONA al campo correcto.   
 3                                                         Marcado de clusters por posible doble unión (sin eliminar filas).   
 
    m_est_cambiados  m_dir_cambiadas  m_tel_cambiados  m_zona_con_valor  \
 0           3047.0              0.0             60.0               NaN   
 1              NaN              NaN         

## 11) Exportar a Excel (múltiples tablas)

In [92]:
with pd.ExcelWriter(EXCEL_XLSX, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="LIMPIO_V4_3")
    if os.path.exists(CAP_CLUSTERS_CSV):
        pd.read_csv(CAP_CLUSTERS_CSV, dtype=str).to_excel(writer, index=False, sheet_name="DUPS_CAPITAL_CLUSTERS")
    if os.path.exists(CAP_DETALLE_CSV):
        pd.read_csv(CAP_DETALLE_CSV, dtype=str).to_excel(writer, index=False, sheet_name="DUPS_CAPITAL_DETALLE")
EXCEL_XLSX

'salidas_proyecto1_v4_3.xlsx'

## 12) Libro de Códigos

In [93]:
def valores_posibles(col, max_items=80):
    if col not in df.columns: return 0, ""
    vals = df[col].dropna().astype(str).unique().tolist()
    vals = sorted(vals)
    n = len(vals)
    show = ", ".join(vals[:max_items]) + (", ..." if n > max_items else "")
    return n, show

lines = []
lines.append("# Libro de Códigos – Establecimientos (Diversificado, v4.3)\n")
lines.append(f"- **Archivo de entrada:** {INPUT_CSV}")

lines.append("## Estándares de limpieza aplicados\n")
lines.append("- Texto en MAYÚSCULAS y SIN ACENTOS en campos categóricos.")
lines.append("- Guiones internos conservados con espacios estándar (` - `).")
lines.append("- Bullets iniciales removidos en `ESTABLECIMIENTO`/`DIRECCION`.")
lines.append("- `DIRECCION`: abreviaturas expandidas; OCR en `RUTA` (I/L→1, O→0).")
lines.append("- `ZONA`: extraída de dirección y/o dept/muni; `CIUDAD CAPITAL` → `GUATEMALA/GUATEMALA`.")
lines.append("- `DIRECCION_STD` con orden canónico: AVENIDA, CALLE, KM, NUMERO, nominativos, resto.")
lines.append("- Duplicados en capital por posible doble unión (`DUP_CAPITAL_MERGE`, `DUP_CAPITAL_CODIGOS_DISTINTOS`).")
lines.append("- Sin eliminación de filas/columnas; solo marcado/normalización.\n")

lines.append("## Catálogos (valores posibles resumidos)\n")
for col in ["DEPARTAMENTO","MUNICIPIO","SECTOR","AREA","STATUS","MODALIDAD","JORNADA","PLAN","NIVEL","ZONA"]:
    if col in df.columns:
        n, show = valores_posibles(col)
        lines.append(f"### {col}\n- **Distintos:** {n}\n- **Ejemplos/Lista:** {show}\n")

lines.append("## Variables (descripción y ejemplos)\n")
for col in df.columns:
    ejemplo = df[col].dropna().astype(str).head(3).tolist()
    ejemplo = [e[:120] for e in ejemplo]
    desc = ""
    if col == "ESTABLECIMIENTO_ORIG": desc = "Valor original del nombre (antes de limpieza)."
    elif col == "DIRECCION_ORIG": desc = "Valor original de la dirección (antes de limpieza)."
    elif col == "TELEFONO_ORIG": desc = "Valor original del teléfono (antes de extracción)."
    elif col == "TELEFONO_VALIDO": desc = "Indicador: True si se extrajo al menos un teléfono de 8 dígitos."
    elif col == "DIRECCION_STD": desc = "Dirección estandarizada (orden canónico)."
    elif col in {"DIR_AVENIDA","DIR_CALLE","DIR_KM","DIR_NUMERO"}: desc = "Componente extraído de la dirección."
    elif col == "ZONA": desc = "Zona administrativa (si aplica); NA si no disponible."
    elif col in {"DEPARTAMENTO","MUNICIPIO"}: desc = "Ubicación administrativa normalizada."
    elif col == "DUP_CAPITAL_MERGE": desc = "True si el registro forma parte de un cluster duplicado en la capital (posible doble unión)."
    elif col == "DUP_CAPITAL_CODIGOS_DISTINTOS": desc = "True si dentro de su cluster hay códigos distintos."
    else: desc = "Campo de la fuente; normalizado cuando aplica."
    lines.append(f"### {col}\n- **Descripción:** {desc}\n- **Ejemplo(s):** {', '.join(ejemplo) if ejemplo else '—'}\n")

with open(CODEBOOK_MD, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

CODEBOOK_MD

'Libro_de_Codigos_Proyecto1_v4_3.md'

## 13) Normalización adicional (NA en admin + categorías finas)

In [94]:
# 13.1 NA estándar en columnas admin
admin_na_cols = ["SUPERVISOR","DISTRITO","DIRECTOR","SECTOR","AREA","STATUS","JORNADA","PLAN","DEPARTAMENTAL"]
placeholders = {"", "nan", "None", "-", "--", "—"}
for col in admin_na_cols:
    if col in df.columns:
        s = df[col].astype(object)
        s = s.where(~s.fillna("").isin(placeholders), pd.NA)
        df[col] = s

# 13.2 Normalización fina de categorías
def cat_standardize(val: str) -> str:
    if not isinstance(val, str) or not val.strip():
        return val
    def _strip_acc(x): 
        return "".join(ch for ch in unicodedata.normalize("NFKD", x) if not unicodedata.combining(ch))
    v = val.strip()
    v = _strip_acc(v).upper()
    v = re.sub(r"\s*-\s*", " - ", v)
    v = re.sub(r"\s*,\s*", ", ", v)
    v = re.sub(r"\s*/\s*", " / ", v)
    v = re.sub(r"(?<=\S)\(", " (", v)
    v = re.sub(r"\(\s+", "(", v)
    v = re.sub(r"\s+\)", ")", v)
    v = re.sub(r"\s{2,}", " ", v).strip()
    return v

cat_cols_fix = ["DEPARTAMENTAL","PLAN","JORNADA","MODALIDAD","STATUS","AREA","SECTOR","NIVEL"]
for col in cat_cols_fix:
    if col in df.columns:
        df[col] = df[col].map(lambda x: cat_standardize(x) if isinstance(x, str) else x)

# 13.3 Recalcular duplicados capital y re-escribir salidas
def _norm2(s): 
    if not isinstance(s, str): 
        return ""
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = s.upper().strip()
    s = re.sub(r"\s*-\s*", " - ", s)
    s = re.sub(r"\s{2,}", " ", s)
    return s

df["__KEY_MERGE_CAP__"] = df.apply(lambda r: "|".join([
    _norm2(r.get("ESTABLECIMIENTO","")),
    _norm2(r.get("DIRECCION_STD","")),
    _norm2(r.get("MUNICIPIO","")),
    _norm2(r.get("DEPARTAMENTO","")),
    _norm2(r.get("JORNADA","")),
    _norm2(r.get("PLAN","")),
    _norm2(r.get("TELEFONO","")),
]), axis=1)

mask_capital = (df.get("DEPARTAMENTO","")=="GUATEMALA") & (df.get("MUNICIPIO","")=="GUATEMALA")
grp_sizes = df[mask_capital].groupby("__KEY_MERGE_CAP__")["__KEY_MERGE_CAP__"].transform("size") if mask_capital.any() else pd.Series([], dtype=int)

df["DUP_CAPITAL_MERGE"] = False
if mask_capital.any():
    df.loc[mask_capital & grp_sizes.gt(1), "DUP_CAPITAL_MERGE"] = True

if df["DUP_CAPITAL_MERGE"].any():
    dup_cap_groups = df.loc[df["DUP_CAPITAL_MERGE"]].groupby("__KEY_MERGE_CAP__")
    key_has_diff_codes = dup_cap_groups["CODIGO"].apply(lambda s: len(set(s.dropna().astype(str))) > 1)
    key_has_diff_codes = key_has_diff_codes.reindex(df["__KEY_MERGE_CAP__"]).fillna(False).values
else:
    key_has_diff_codes = [False]*len(df)
df["DUP_CAPITAL_CODIGOS_DISTINTOS"] = key_has_diff_codes

df.to_csv(CLEAN_CSV, index=False, encoding="utf-8")

cap_dups = df.loc[df["DUP_CAPITAL_MERGE"], ["__KEY_MERGE_CAP__","CODIGO","ESTABLECIMIENTO","DIRECCION_STD","ZONA","TELEFONO","JORNADA","PLAN","MUNICIPIO","DEPARTAMENTO","DUP_CAPITAL_CODIGOS_DISTINTOS"]]
cluster_rows = []
for key, sub in cap_dups.groupby("__KEY_MERGE_CAP__"):
    cods = sorted(set(sub["CODIGO"].dropna().astype(str)))
    cluster_rows.append({
        "KEY": key,
        "ESTABLECIMIENTO": sub["ESTABLECIMIENTO"].iloc[0],
        "DIRECCION_STD": sub["DIRECCION_STD"].iloc[0],
        "ZONA": sub["ZONA"].iloc[0],
        "JORNADA": sub["JORNADA"].iloc[0],
        "PLAN": sub["PLAN"].iloc[0],
        "TELEFONO": sub["TELEFONO"].iloc[0],
        "MUNICIPIO": sub["MUNICIPIO"].iloc[0],
        "DEPARTAMENTO": sub["DEPARTAMENTO"].iloc[0],
        "N_FILAS": len(sub),
        "CODIGOS": " | ".join(cods),
        "CODIGOS_DISTINTOS": (len(cods) > 1)
    })
cap_clusters = pd.DataFrame(cluster_rows).sort_values(["ESTABLECIMIENTO","ZONA","DIRECCION_STD","JORNADA","PLAN"]) if cluster_rows else pd.DataFrame(columns=["KEY","ESTABLECIMIENTO","DIRECCION_STD","ZONA","JORNADA","PLAN","TELEFONO","MUNICIPIO","DEPARTAMENTO","N_FILAS","CODIGOS","CODIGOS_DISTINTOS"])
cap_clusters.to_csv(CAP_CLUSTERS_CSV, index=False, encoding="utf-8")
cap_dups.drop(columns=["__KEY_MERGE_CAP__"]).to_csv(CAP_DETALLE_CSV, index=False, encoding="utf-8")

CLEAN_CSV, CAP_CLUSTERS_CSV, CAP_DETALLE_CSV

C:\Users\ricar\AppData\Local\Temp\ipykernel_18336\3354604961.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  key_has_diff_codes = key_has_diff_codes.reindex(df["__KEY_MERGE_CAP__"]).fillna(False).values


('establecimientos_diversificado_limpio.csv',
 'duplicados_capital_merge_v4.csv',
 'duplicados_capital_merge_detalle_v4.csv')

## 14) Re-generar Excel y Libro de Códigos (post-normalización, rutas relativas)

In [95]:
# Libro de Códigos
def valores_posibles(df, col, max_items=80):
    if col not in df.columns: return 0, ""
    vals = df[col].dropna().astype(str).unique().tolist()
    vals = sorted(vals)
    n = len(vals)
    show = ", ".join(vals[:max_items]) + (", ..." if n > max_items else "")
    return n, show

lines = []
lines.append("# Libro de Códigos – Establecimientos (Diversificado)\n")
lines.append(f"- **Archivo de salida base:** {CLEAN_CSV}")

lines.append("## Estándares de limpieza aplicados (resumen)\n")
lines.append("- NA estandarizado en columnas administrativas (SUPERVISOR, DISTRITO, DIRECTOR, SECTOR, AREA, STATUS, JORNADA, PLAN, DEPARTAMENTAL).")
lines.append("- Normalización fina de categorías (espacios/paréntesis/guiones) para evitar duplicados aparentes.")
lines.append("- Re-cálculo de duplicados en capital con categorías ya normalizadas.\n")

lines.append("## Catálogos (valores posibles, resumido)\n")
for col in ["DEPARTAMENTO","MUNICIPIO","SECTOR","AREA","STATUS","MODALIDAD","JORNADA","PLAN","NIVEL","ZONA","DEPARTAMENTAL"]:
    if col in df.columns:
        n, show = valores_posibles(df, col)
        lines.append(f"### {col}\n- **Distintos:** {n}\n- **Ejemplos/Lista:** {show}\n")

lines.append("## Variables (descripción breve)\n")
for col in df.columns:
    desc = ""
    if col == "ESTABLECIMIENTO_ORIG": desc = "Valor original del nombre (antes de limpieza)."
    elif col == "DIRECCION_ORIG": desc = "Valor original de la dirección (antes de limpieza)."
    elif col == "TELEFONO_ORIG": desc = "Valor original del teléfono (antes de extracción)."
    elif col == "TELEFONO": desc = "Teléfonos de 8 dígitos extraídos y deduplicados; NA si no hay válido."
    elif col == "TELEFONO_VALIDO": desc = "Indicador: True si `TELEFONO` no es NA."
    elif col == "DIRECCION_STD": desc = "Dirección estandarizada (orden canónico)."
    elif col in {"DIR_AVENIDA","DIR_CALLE","DIR_KM","DIR_NUMERO"}: desc = "Componente extraído de la dirección."
    elif col == "ZONA": desc = "Zona administrativa (si aplica); NA si no disponible."
    elif col in {"DEPARTAMENTO","MUNICIPIO"}: desc = "Ubicación administrativa normalizada."
    elif col == "DUP_CAPITAL_MERGE": desc = "True si el registro forma parte de un cluster duplicado en la capital (posible doble unión)."
    elif col == "DUP_CAPITAL_CODIGOS_DISTINTOS": desc = "True si dentro de su cluster hay códigos distintos."
    else: desc = "Campo de la fuente; normalizado cuando aplica."
    lines.append(f"- **{col}**: {desc}")
with open(CODEBOOK_MD, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

# Excel v4.3 (rutas relativas)
with pd.ExcelWriter(EXCEL_XLSX, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="LIMPIO_V4_3")
    if os.path.exists(CAP_CLUSTERS_CSV):
        pd.read_csv(CAP_CLUSTERS_CSV, dtype=str).to_excel(writer, index=False, sheet_name="DUPS_CAPITAL_CLUSTERS")
    if os.path.exists(CAP_DETALLE_CSV):
        pd.read_csv(CAP_DETALLE_CSV, dtype=str).to_excel(writer, index=False, sheet_name="DUPS_CAPITAL_DETALLE")

CODEBOOK_MD, EXCEL_XLSX

('Libro_de_Codigos_Proyecto1_v4_3.md', 'salidas_proyecto1_v4_3.xlsx')